In [13]:
model_dir="/projects/tir4/users/pfernand/workflows-outputs/metaexpl-experiments/v2/TrainTeacher/Baseline.baseline/teacher_dir"

# list of explainers to load of the form ´explainer: save_params_path´. if no path is provided
# it will initialize the explainer
explainers_dirs = {
    "attention_softmax_raw": "/projects/tir4/users/pfernand/workflows-outputs/metaexpl-experiments/v2/TrainStudent/KLDCoeff.1+NumExamples.2000/teacher_expl_dir/",
    "attention_softmax_trained": "/projects/tir4/users/pfernand/workflows-outputs/metaexpl-experiments/v2/TrainStudent/KLDCoeff.1+MetaInterval.1+MetaLR.5e-4+MetaLearn.true+NumExamples.2000+Seed.1+TeacherExplainer.attention/teacher_expl_dir/",
    "hidden_qk_softmax_raw": "/projects/tir4/users/pfernand/workflows-outputs/metaexpl-experiments/v2/TrainStudent/NumExamples.2000+Seed.1+TeacherExplainer.hidden_qk/teacher_expl_dir/",
    "hidden_qk_softmax_trained": "/projects/tir4/users/pfernand/workflows-outputs/metaexpl-experiments/v2/TrainStudent/KLDCoeff.1+MetaInterval.1+MetaLR.5e-4+MetaLearn.true+NumExamples.2000+Seed.1+TeacherExplainer.hidden_qk/teacher_expl_dir/",
    "attention_qk_softmax_raw": "/projects/tir4/users/pfernand/workflows-outputs/metaexpl-experiments/v2/TrainStudent/NumExamples.2000+Seed.1+TeacherExplainer.attention_query/teacher_expl_dir/",
    "attention_qk_softmax_trained": "/projects/tir4/users/pfernand/workflows-outputs/metaexpl-experiments/v2/TrainStudent/KLDCoeff.1+MetaInterval.1+MetaLR.5e-4+MetaLearn.true+NumExamples.2000+Seed.1+TeacherExplainer.attention_query/teacher_expl_dir/",
}
seed=0
max_len=256
batch_size=1

In [14]:
%load_ext autoreload
%autoreload 2
from meta_expl.models import load_model
from meta_expl.explainers import *
from meta_expl.utils import PRNGSequence
from meta_expl.data import load_data

import jax.numpy as jnp

from transformers import ElectraTokenizerFast

rngseq = PRNGSequence(seed)

tokenizer = ElectraTokenizerFast.from_pretrained("google/electra-small-discriminator")

model, params, dummy_state = load_model(model_dir, batch_size=1)
input_ids = jnp.ones((batch_size, max_len), jnp.int32)
dummy_inputs = {
    "input_ids": input_ids,
    "attention_mask": jnp.ones_like(input_ids),
    "token_type_ids": jnp.arange(jnp.atleast_2d(input_ids).shape[-1]),
    "position_ids": jnp.ones_like(input_ids),
}

explainers = {}
for explainer_name, explainer_dir in explainers_dirs.items():
    if explainer_dir is None:
        explainers[explainer_name] = create_explainer(next(rngseq), dummy_state, explainer_type=explainer_name)
    else:
        explainers[explainer_name] = load_explainer(explainer_dir, dummy_inputs, dummy_state)
        print(f"{explainer_name}:", explainers[explainer_name][1]["params"])

data = load_data("imdb", "student", "test")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
{'explainer_args': {'normalizer_fn': 'softmax', 'aggregator_idx': 0, 'layer_idx': -1, 'init_fn': 'uniform', 'parent': None, 'name': None}, 'explainer_type': 'attention_explainer'}
attention_softmax_raw: FrozenDict({
    head_coeffs: array([0.25, 0.25, 0.25, 0.25], dtype=float32),
})
{'explainer_args': {'normalizer_fn': 'softmax', 'aggregator_idx': 0, 'layer_idx': -1, 'init_fn': 'uniform', 'parent': None, 'name': None}, 'explainer_type': 'attention_explainer'}
attention_softmax_trained: FrozenDict({
    head_coeffs: array([0.22792691, 0.1683412 , 0.27172807, 0.17008352], dtype=float32),
})
{'explainer_args': {'normalizer_fn': 'softmax', 'layer_idx': -1, 'kq_dim': 1028, 'parent': None, 'name': None}, 'explainer_type': 'hidden_qk_explainer'}
hidden_qk_softmax_raw: FrozenDict({
    Dense_0: {
        kernel: array([[ 0.08950771,  0.03103238,  0.08137495, ..., -0.07013398,
                 0.03273486,  0

In [11]:
from IPython.display import Markdown, display

def print_coloured(subwords, weights, max_alpha=0.8):
    text = []
    colour = ["250, 133, 205" if w <= 0 else "135,206,250" for w in weights]
    weights = weights / jnp.max(weights)
    for i, subword in enumerate(subwords):
        if subword[0] == "▁":
            subword = subword[1:]
            if i:
                text.append(" ")
        colorstr = f"<span style='background-color:rgba({colour[i]},{weights[i] / max_alpha:.03f}'>{subword}</span>"
        text.append(colorstr)
    display(Markdown(" ".join(text)))

In [15]:
import jax
import jax.numpy as jnp
from functools import partial

@partial(jax.jit, static_argnums=(0,))
def get_state(model, params, x):
    """ Evaluation step """
    return model.apply(params, **x)[1]

@partial(jax.jit, static_argnums=(0,))
def get_explanation(explainer, params, x, state):
    """ Evaluation step """
    return explainer.apply(
        params, x, state
    )[0]

num_samples = 20
for sample in data[:num_samples]:
    label, inputs = sample.values()
    # extract token ids and subwords from tokenizer
    x = dict(tokenizer([inputs], truncation=True, padding='max_length', max_length=max_len, return_tensors="jax"))
    num_tokens = jnp.sum(jnp.where(x['input_ids'] != 0, 1, 0))
    bp_tokens = tokenizer.convert_ids_to_tokens(x['input_ids'][0][:num_tokens])
    # extract model state 
    state = get_state(model, params, x)
    print(f"Label: {label}\n")
    for explainer, (expl_cls, expl_params) in explainers.items():
        print(f"Explainer: {explainer}")
        explanation = get_explanation(expl_cls, expl_params, x, state)[0][:num_tokens]
        assert jnp.isclose(jnp.sum(explanation), 1), jnp.sum(explanation)
        print_coloured(bp_tokens, explanation)
    print("----------")

Label: 1

Explainer: attention_softmax_raw


<span style='background-color:rgba(135,206,250,0.233'>[CLS]</span> <span style='background-color:rgba(135,206,250,0.344'>four</span> <span style='background-color:rgba(135,206,250,0.134'>stories</span> <span style='background-color:rgba(135,206,250,0.022'>about</span> <span style='background-color:rgba(135,206,250,0.008'>the</span> <span style='background-color:rgba(135,206,250,0.002'>drug</span> <span style='background-color:rgba(135,206,250,0.001'>trade</span> <span style='background-color:rgba(135,206,250,0.001'>in</span> <span style='background-color:rgba(135,206,250,0.004'>europe</span> <span style='background-color:rgba(135,206,250,0.371'>become</span> <span style='background-color:rgba(135,206,250,0.303'>inter</span> <span style='background-color:rgba(135,206,250,0.194'>##twined</span> <span style='background-color:rgba(135,206,250,0.093'>over</span> <span style='background-color:rgba(135,206,250,0.152'>the</span> <span style='background-color:rgba(135,206,250,0.163'>course</span> <span style='background-color:rgba(135,206,250,0.072'>of</span> <span style='background-color:rgba(135,206,250,0.248'>this</span> <span style='background-color:rgba(135,206,250,0.327'>6</span> <span style='background-color:rgba(135,206,250,0.305'>part</span> <span style='background-color:rgba(135,206,250,0.654'>miniseries</span> <span style='background-color:rgba(135,206,250,0.071'>.</span> <span style='background-color:rgba(135,206,250,0.002'>in</span> <span style='background-color:rgba(135,206,250,0.003'>germany</span> <span style='background-color:rgba(135,206,250,0.004'>a</span> <span style='background-color:rgba(135,206,250,0.001'>businessman</span> <span style='background-color:rgba(135,206,250,0.002'>is</span> <span style='background-color:rgba(135,206,250,0.000'>arrested</span> <span style='background-color:rgba(135,206,250,0.000'>on</span> <span style='background-color:rgba(135,206,250,0.001'>drug</span> <span style='background-color:rgba(135,206,250,0.001'>smuggling</span> <span style='background-color:rgba(135,206,250,0.001'>charges</span> <span style='background-color:rgba(135,206,250,0.001'>,</span> <span style='background-color:rgba(135,206,250,0.001'>and</span> <span style='background-color:rgba(135,206,250,0.001'>his</span> <span style='background-color:rgba(135,206,250,0.001'>wife</span> <span style='background-color:rgba(135,206,250,0.000'>attempts</span> <span style='background-color:rgba(135,206,250,0.000'>to</span> <span style='background-color:rgba(135,206,250,0.000'>save</span> <span style='background-color:rgba(135,206,250,0.000'>her</span> <span style='background-color:rgba(135,206,250,0.000'>family</span> <span style='background-color:rgba(135,206,250,0.001'>by</span> <span style='background-color:rgba(135,206,250,0.001'>continuing</span> <span style='background-color:rgba(135,206,250,0.000'>her</span> <span style='background-color:rgba(135,206,250,0.001'>husband</span> <span style='background-color:rgba(135,206,250,0.039'>'</span> <span style='background-color:rgba(135,206,250,0.001'>s</span> <span style='background-color:rgba(135,206,250,0.001'>illegal</span> <span style='background-color:rgba(135,206,250,0.000'>trade</span> <span style='background-color:rgba(135,206,250,0.003'>.</span> <span style='background-color:rgba(135,206,250,0.006'>meanwhile</span> <span style='background-color:rgba(135,206,250,0.007'>the</span> <span style='background-color:rgba(135,206,250,0.006'>british</span> <span style='background-color:rgba(135,206,250,0.002'>home</span> <span style='background-color:rgba(135,206,250,0.001'>secretary</span> <span style='background-color:rgba(135,206,250,0.001'>travels</span> <span style='background-color:rgba(135,206,250,0.000'>to</span> <span style='background-color:rgba(135,206,250,0.001'>pakistan</span> <span style='background-color:rgba(135,206,250,0.001'>to</span> <span style='background-color:rgba(135,206,250,0.002'>negotiate</span> <span style='background-color:rgba(135,206,250,0.002'>an</span> <span style='background-color:rgba(135,206,250,0.001'>aid</span> <span style='background-color:rgba(135,206,250,0.001'>package</span> <span style='background-color:rgba(135,206,250,0.001'>that</span> <span style='background-color:rgba(135,206,250,0.000'>he</span> <span style='background-color:rgba(135,206,250,0.001'>hopes</span> <span style='background-color:rgba(135,206,250,0.001'>will</span> <span style='background-color:rgba(135,206,250,0.000'>stop</span> <span style='background-color:rgba(135,206,250,0.001'>the</span> <span style='background-color:rgba(135,206,250,0.000'>drug</span> <span style='background-color:rgba(135,206,250,0.000'>flow</span> <span style='background-color:rgba(135,206,250,0.000'>from</span> <span style='background-color:rgba(135,206,250,0.000'>that</span> <span style='background-color:rgba(135,206,250,0.000'>country</span> <span style='background-color:rgba(135,206,250,0.010'>.</span> <span style='background-color:rgba(135,206,250,0.029'>even</span> <span style='background-color:rgba(135,206,250,0.006'>as</span> <span style='background-color:rgba(135,206,250,0.001'>he</span> <span style='background-color:rgba(135,206,250,0.003'>does</span> <span style='background-color:rgba(135,206,250,0.025'>this</span> <span style='background-color:rgba(135,206,250,0.028'>however</span> <span style='background-color:rgba(135,206,250,0.005'>his</span> <span style='background-color:rgba(135,206,250,0.007'>own</span> <span style='background-color:rgba(135,206,250,0.003'>daughter</span> <span style='background-color:rgba(135,206,250,0.007'>su</span> <span style='background-color:rgba(135,206,250,0.004'>##cc</span> <span style='background-color:rgba(135,206,250,0.003'>##umb</span> <span style='background-color:rgba(135,206,250,0.018'>##s</span> <span style='background-color:rgba(135,206,250,0.001'>to</span> <span style='background-color:rgba(135,206,250,0.002'>heroin</span> <span style='background-color:rgba(135,206,250,0.001'>addiction</span> <span style='background-color:rgba(135,206,250,0.018'>,</span> <span style='background-color:rgba(135,206,250,0.036'>tearing</span> <span style='background-color:rgba(135,206,250,0.002'>their</span> <span style='background-color:rgba(135,206,250,0.002'>family</span> <span style='background-color:rgba(135,206,250,0.007'>apart</span> <span style='background-color:rgba(135,206,250,0.044'>.</span> <span style='background-color:rgba(135,206,250,0.670'>this</span> <span style='background-color:rgba(135,206,250,1.250'>is</span> <span style='background-color:rgba(135,206,250,0.451'>the</span> <span style='background-color:rgba(135,206,250,0.957'>miniseries</span> <span style='background-color:rgba(135,206,250,0.080'>on</span> <span style='background-color:rgba(135,206,250,0.054'>which</span> <span style='background-color:rgba(135,206,250,0.021'>the</span> <span style='background-color:rgba(135,206,250,0.008'>american</span> <span style='background-color:rgba(135,206,250,0.010'>film</span> <span style='background-color:rgba(135,206,250,0.004'>and</span> <span style='background-color:rgba(135,206,250,0.007'>miniseries</span> <span style='background-color:rgba(135,206,250,0.008'>were</span> <span style='background-color:rgba(135,206,250,0.013'>based</span> <span style='background-color:rgba(135,206,250,0.102'>.</span> <span style='background-color:rgba(135,206,250,0.174'>the</span> <span style='background-color:rgba(135,206,250,0.074'>original</span> <span style='background-color:rgba(135,206,250,0.373'>is</span> <span style='background-color:rgba(135,206,250,0.185'>far</span> <span style='background-color:rgba(135,206,250,0.181'>superior</span> <span style='background-color:rgba(135,206,250,0.014'>to</span> <span style='background-color:rgba(135,206,250,0.038'>its</span> <span style='background-color:rgba(135,206,250,0.011'>two</span> <span style='background-color:rgba(135,206,250,0.014'>dec</span> <span style='background-color:rgba(135,206,250,0.003'>##end</span> <span style='background-color:rgba(135,206,250,0.004'>##ants</span> <span style='background-color:rgba(135,206,250,0.065'>.</span> <span style='background-color:rgba(135,206,250,0.141'>the</span> <span style='background-color:rgba(135,206,250,0.010'>poverty</span> <span style='background-color:rgba(135,206,250,0.032'>and</span> <span style='background-color:rgba(135,206,250,0.023'>desperation</span> <span style='background-color:rgba(135,206,250,0.010'>of</span> <span style='background-color:rgba(135,206,250,0.012'>the</span> <span style='background-color:rgba(135,206,250,0.011'>third</span> <span style='background-color:rgba(135,206,250,0.016'>world</span> <span style='background-color:rgba(135,206,250,0.183'>are</span> <span style='background-color:rgba(135,206,250,0.081'>portrayed</span> <span style='background-color:rgba(135,206,250,0.467'>very</span> <span style='background-color:rgba(135,206,250,0.169'>well</span> <span style='background-color:rgba(135,206,250,0.197'>.</span> <span style='background-color:rgba(135,206,250,0.646'>it</span> <span style='background-color:rgba(135,206,250,0.902'>is</span> <span style='background-color:rgba(135,206,250,0.499'>stark</span> <span style='background-color:rgba(135,206,250,0.326'>,</span> <span style='background-color:rgba(135,206,250,0.447'>un</span> <span style='background-color:rgba(135,206,250,0.088'>##com</span> <span style='background-color:rgba(135,206,250,0.206'>##promising</span> <span style='background-color:rgba(135,206,250,0.319'>and</span> <span style='background-color:rgba(135,206,250,0.498'>brutal</span> <span style='background-color:rgba(135,206,250,0.425'>.</span> <span style='background-color:rgba(135,206,250,0.233'>[SEP]</span>

Explainer: attention_softmax_trained


<span style='background-color:rgba(135,206,250,0.324'>[CLS]</span> <span style='background-color:rgba(135,206,250,0.555'>four</span> <span style='background-color:rgba(135,206,250,0.243'>stories</span> <span style='background-color:rgba(135,206,250,0.053'>about</span> <span style='background-color:rgba(135,206,250,0.019'>the</span> <span style='background-color:rgba(135,206,250,0.007'>drug</span> <span style='background-color:rgba(135,206,250,0.004'>trade</span> <span style='background-color:rgba(135,206,250,0.003'>in</span> <span style='background-color:rgba(135,206,250,0.012'>europe</span> <span style='background-color:rgba(135,206,250,0.530'>become</span> <span style='background-color:rgba(135,206,250,0.495'>inter</span> <span style='background-color:rgba(135,206,250,0.343'>##twined</span> <span style='background-color:rgba(135,206,250,0.169'>over</span> <span style='background-color:rgba(135,206,250,0.251'>the</span> <span style='background-color:rgba(135,206,250,0.244'>course</span> <span style='background-color:rgba(135,206,250,0.135'>of</span> <span style='background-color:rgba(135,206,250,0.376'>this</span> <span style='background-color:rgba(135,206,250,0.583'>6</span> <span style='background-color:rgba(135,206,250,0.537'>part</span> <span style='background-color:rgba(135,206,250,0.923'>miniseries</span> <span style='background-color:rgba(135,206,250,0.134'>.</span> <span style='background-color:rgba(135,206,250,0.007'>in</span> <span style='background-color:rgba(135,206,250,0.008'>germany</span> <span style='background-color:rgba(135,206,250,0.013'>a</span> <span style='background-color:rgba(135,206,250,0.005'>businessman</span> <span style='background-color:rgba(135,206,250,0.006'>is</span> <span style='background-color:rgba(135,206,250,0.002'>arrested</span> <span style='background-color:rgba(135,206,250,0.002'>on</span> <span style='background-color:rgba(135,206,250,0.005'>drug</span> <span style='background-color:rgba(135,206,250,0.003'>smuggling</span> <span style='background-color:rgba(135,206,250,0.003'>charges</span> <span style='background-color:rgba(135,206,250,0.005'>,</span> <span style='background-color:rgba(135,206,250,0.003'>and</span> <span style='background-color:rgba(135,206,250,0.003'>his</span> <span style='background-color:rgba(135,206,250,0.002'>wife</span> <span style='background-color:rgba(135,206,250,0.002'>attempts</span> <span style='background-color:rgba(135,206,250,0.002'>to</span> <span style='background-color:rgba(135,206,250,0.002'>save</span> <span style='background-color:rgba(135,206,250,0.001'>her</span> <span style='background-color:rgba(135,206,250,0.002'>family</span> <span style='background-color:rgba(135,206,250,0.002'>by</span> <span style='background-color:rgba(135,206,250,0.003'>continuing</span> <span style='background-color:rgba(135,206,250,0.002'>her</span> <span style='background-color:rgba(135,206,250,0.002'>husband</span> <span style='background-color:rgba(135,206,250,0.076'>'</span> <span style='background-color:rgba(135,206,250,0.002'>s</span> <span style='background-color:rgba(135,206,250,0.003'>illegal</span> <span style='background-color:rgba(135,206,250,0.002'>trade</span> <span style='background-color:rgba(135,206,250,0.010'>.</span> <span style='background-color:rgba(135,206,250,0.017'>meanwhile</span> <span style='background-color:rgba(135,206,250,0.019'>the</span> <span style='background-color:rgba(135,206,250,0.015'>british</span> <span style='background-color:rgba(135,206,250,0.008'>home</span> <span style='background-color:rgba(135,206,250,0.004'>secretary</span> <span style='background-color:rgba(135,206,250,0.003'>travels</span> <span style='background-color:rgba(135,206,250,0.002'>to</span> <span style='background-color:rgba(135,206,250,0.004'>pakistan</span> <span style='background-color:rgba(135,206,250,0.003'>to</span> <span style='background-color:rgba(135,206,250,0.006'>negotiate</span> <span style='background-color:rgba(135,206,250,0.006'>an</span> <span style='background-color:rgba(135,206,250,0.005'>aid</span> <span style='background-color:rgba(135,206,250,0.004'>package</span> <span style='background-color:rgba(135,206,250,0.003'>that</span> <span style='background-color:rgba(135,206,250,0.002'>he</span> <span style='background-color:rgba(135,206,250,0.002'>hopes</span> <span style='background-color:rgba(135,206,250,0.003'>will</span> <span style='background-color:rgba(135,206,250,0.001'>stop</span> <span style='background-color:rgba(135,206,250,0.003'>the</span> <span style='background-color:rgba(135,206,250,0.002'>drug</span> <span style='background-color:rgba(135,206,250,0.001'>flow</span> <span style='background-color:rgba(135,206,250,0.001'>from</span> <span style='background-color:rgba(135,206,250,0.001'>that</span> <span style='background-color:rgba(135,206,250,0.001'>country</span> <span style='background-color:rgba(135,206,250,0.025'>.</span> <span style='background-color:rgba(135,206,250,0.060'>even</span> <span style='background-color:rgba(135,206,250,0.017'>as</span> <span style='background-color:rgba(135,206,250,0.004'>he</span> <span style='background-color:rgba(135,206,250,0.007'>does</span> <span style='background-color:rgba(135,206,250,0.048'>this</span> <span style='background-color:rgba(135,206,250,0.050'>however</span> <span style='background-color:rgba(135,206,250,0.012'>his</span> <span style='background-color:rgba(135,206,250,0.015'>own</span> <span style='background-color:rgba(135,206,250,0.010'>daughter</span> <span style='background-color:rgba(135,206,250,0.018'>su</span> <span style='background-color:rgba(135,206,250,0.013'>##cc</span> <span style='background-color:rgba(135,206,250,0.009'>##umb</span> <span style='background-color:rgba(135,206,250,0.043'>##s</span> <span style='background-color:rgba(135,206,250,0.003'>to</span> <span style='background-color:rgba(135,206,250,0.006'>heroin</span> <span style='background-color:rgba(135,206,250,0.004'>addiction</span> <span style='background-color:rgba(135,206,250,0.041'>,</span> <span style='background-color:rgba(135,206,250,0.082'>tearing</span> <span style='background-color:rgba(135,206,250,0.005'>their</span> <span style='background-color:rgba(135,206,250,0.007'>family</span> <span style='background-color:rgba(135,206,250,0.019'>apart</span> <span style='background-color:rgba(135,206,250,0.084'>.</span> <span style='background-color:rgba(135,206,250,0.777'>this</span> <span style='background-color:rgba(135,206,250,1.250'>is</span> <span style='background-color:rgba(135,206,250,0.616'>the</span> <span style='background-color:rgba(135,206,250,1.213'>miniseries</span> <span style='background-color:rgba(135,206,250,0.141'>on</span> <span style='background-color:rgba(135,206,250,0.094'>which</span> <span style='background-color:rgba(135,206,250,0.043'>the</span> <span style='background-color:rgba(135,206,250,0.020'>american</span> <span style='background-color:rgba(135,206,250,0.021'>film</span> <span style='background-color:rgba(135,206,250,0.010'>and</span> <span style='background-color:rgba(135,206,250,0.017'>miniseries</span> <span style='background-color:rgba(135,206,250,0.019'>were</span> <span style='background-color:rgba(135,206,250,0.032'>based</span> <span style='background-color:rgba(135,206,250,0.171'>.</span> <span style='background-color:rgba(135,206,250,0.276'>the</span> <span style='background-color:rgba(135,206,250,0.144'>original</span> <span style='background-color:rgba(135,206,250,0.464'>is</span> <span style='background-color:rgba(135,206,250,0.309'>far</span> <span style='background-color:rgba(135,206,250,0.292'>superior</span> <span style='background-color:rgba(135,206,250,0.031'>to</span> <span style='background-color:rgba(135,206,250,0.063'>its</span> <span style='background-color:rgba(135,206,250,0.029'>two</span> <span style='background-color:rgba(135,206,250,0.036'>dec</span> <span style='background-color:rgba(135,206,250,0.011'>##end</span> <span style='background-color:rgba(135,206,250,0.011'>##ants</span> <span style='background-color:rgba(135,206,250,0.120'>.</span> <span style='background-color:rgba(135,206,250,0.227'>the</span> <span style='background-color:rgba(135,206,250,0.026'>poverty</span> <span style='background-color:rgba(135,206,250,0.067'>and</span> <span style='background-color:rgba(135,206,250,0.053'>desperation</span> <span style='background-color:rgba(135,206,250,0.025'>of</span> <span style='background-color:rgba(135,206,250,0.029'>the</span> <span style='background-color:rgba(135,206,250,0.028'>third</span> <span style='background-color:rgba(135,206,250,0.034'>world</span> <span style='background-color:rgba(135,206,250,0.266'>are</span> <span style='background-color:rgba(135,206,250,0.147'>portrayed</span> <span style='background-color:rgba(135,206,250,0.666'>very</span> <span style='background-color:rgba(135,206,250,0.259'>well</span> <span style='background-color:rgba(135,206,250,0.301'>.</span> <span style='background-color:rgba(135,206,250,0.744'>it</span> <span style='background-color:rgba(135,206,250,1.008'>is</span> <span style='background-color:rgba(135,206,250,0.681'>stark</span> <span style='background-color:rgba(135,206,250,0.439'>,</span> <span style='background-color:rgba(135,206,250,0.600'>un</span> <span style='background-color:rgba(135,206,250,0.158'>##com</span> <span style='background-color:rgba(135,206,250,0.313'>##promising</span> <span style='background-color:rgba(135,206,250,0.406'>and</span> <span style='background-color:rgba(135,206,250,0.670'>brutal</span> <span style='background-color:rgba(135,206,250,0.549'>.</span> <span style='background-color:rgba(135,206,250,0.325'>[SEP]</span>

Explainer: hidden_qk_softmax_raw


...

Explainer: attention_qk_softmax_raw


<span style='background-color:rgba(135,206,250,1.250'>[CLS]</span> <span style='background-color:rgba(135,206,250,0.003'>first</span> <span style='background-color:rgba(135,206,250,0.008'>off</span> <span style='background-color:rgba(135,206,250,0.002'>,</span> <span style='background-color:rgba(135,206,250,0.005'>i</span> <span style='background-color:rgba(135,206,250,0.005'>have</span> <span style='background-color:rgba(135,206,250,0.002'>been</span> <span style='background-color:rgba(135,206,250,0.002'>a</span> <span style='background-color:rgba(135,206,250,0.003'>fan</span> <span style='background-color:rgba(135,206,250,0.001'>of</span> <span style='background-color:rgba(135,206,250,0.001'>the</span> <span style='background-color:rgba(135,206,250,0.002'>show</span> <span style='background-color:rgba(135,206,250,0.002'>back</span> <span style='background-color:rgba(135,206,250,0.001'>when</span> <span style='background-color:rgba(135,206,250,0.001'>my</span> <span style='background-color:rgba(135,206,250,0.003'>pbs</span> <span style='background-color:rgba(135,206,250,0.002'>station</span> <span style='background-color:rgba(135,206,250,0.003'>started</span> <span style='background-color:rgba(135,206,250,0.002'>showing</span> <span style='background-color:rgba(135,206,250,0.001'>it</span> <span style='background-color:rgba(135,206,250,0.002'>back</span> <span style='background-color:rgba(135,206,250,0.002'>in</span> <span style='background-color:rgba(135,206,250,0.004'>1981</span> <span style='background-color:rgba(135,206,250,0.004'>.</span> <span style='background-color:rgba(135,206,250,0.003'>i</span> <span style='background-color:rgba(135,206,250,0.002'>learned</span> <span style='background-color:rgba(135,206,250,0.004'>many</span> <span style='background-color:rgba(135,206,250,0.002'>things</span> <span style='background-color:rgba(135,206,250,0.001'>about</span> <span style='background-color:rgba(135,206,250,0.001'>the</span> <span style='background-color:rgba(135,206,250,0.001'>show</span> <span style='background-color:rgba(135,206,250,0.001'>and</span> <span style='background-color:rgba(135,206,250,0.001'>the</span> <span style='background-color:rgba(135,206,250,0.001'>people</span> <span style='background-color:rgba(135,206,250,0.000'>who</span> <span style='background-color:rgba(135,206,250,0.000'>were</span> <span style='background-color:rgba(135,206,250,0.001'>in</span> <span style='background-color:rgba(135,206,250,0.002'>or</span> <span style='background-color:rgba(135,206,250,0.002'>contributed</span> <span style='background-color:rgba(135,206,250,0.001'>to</span> <span style='background-color:rgba(135,206,250,0.001'>the</span> <span style='background-color:rgba(135,206,250,0.003'>show</span> <span style='background-color:rgba(135,206,250,0.011'>.</span> <span style='background-color:rgba(135,206,250,0.018'>this</span> <span style='background-color:rgba(135,206,250,0.007'>latest</span> <span style='background-color:rgba(135,206,250,0.008'>installment</span> <span style='background-color:rgba(135,206,250,0.002'>of</span> <span style='background-color:rgba(135,206,250,0.013'>doctor</span> <span style='background-color:rgba(135,206,250,0.010'>who</span> <span style='background-color:rgba(135,206,250,0.010'>made</span> <span style='background-color:rgba(135,206,250,0.007'>a</span> <span style='background-color:rgba(135,206,250,0.021'>great</span> <span style='background-color:rgba(135,206,250,0.017'>impression</span> <span style='background-color:rgba(135,206,250,0.006'>on</span> <span style='background-color:rgba(135,206,250,0.009'>me</span> <span style='background-color:rgba(135,206,250,0.009'>.</span> <span style='background-color:rgba(135,206,250,0.001'>the</span> <span style='background-color:rgba(135,206,250,0.002'>original</span> <span style='background-color:rgba(135,206,250,0.002'>series</span> <span style='background-color:rgba(135,206,250,0.001'>,</span> <span style='background-color:rgba(135,206,250,0.002'>aka</span> <span style='background-color:rgba(135,206,250,0.001'>classic</span> <span style='background-color:rgba(135,206,250,0.003'>series</span> <span style='background-color:rgba(135,206,250,0.001'>,</span> <span style='background-color:rgba(135,206,250,0.002'>was</span> <span style='background-color:rgba(135,206,250,0.003'>made</span> <span style='background-color:rgba(135,206,250,0.002'>fun</span> <span style='background-color:rgba(135,206,250,0.003'>of</span> <span style='background-color:rgba(135,206,250,0.001'>by</span> <span style='background-color:rgba(135,206,250,0.001'>the</span> <span style='background-color:rgba(135,206,250,0.004'>bad</span> <span style='background-color:rgba(135,206,250,0.001'>special</span> <span style='background-color:rgba(135,206,250,0.002'>effects</span> <span style='background-color:rgba(135,206,250,0.001'>and</span> <span style='background-color:rgba(135,206,250,0.008'>/</span> <span style='background-color:rgba(135,206,250,0.001'>or</span> <span style='background-color:rgba(135,206,250,0.001'>wo</span> <span style='background-color:rgba(135,206,250,0.002'>##bbly</span> <span style='background-color:rgba(135,206,250,0.001'>sets</span> <span style='background-color:rgba(135,206,250,0.006'>.</span> <span style='background-color:rgba(135,206,250,0.023'>well</span> <span style='background-color:rgba(135,206,250,0.004'>,</span> <span style='background-color:rgba(135,206,250,0.006'>this</span> <span style='background-color:rgba(135,206,250,0.003'>is</span> <span style='background-color:rgba(135,206,250,0.027'>no</span> <span style='background-color:rgba(135,206,250,0.003'>more</span> <span style='background-color:rgba(135,206,250,0.008'>true</span> <span style='background-color:rgba(135,206,250,0.003'>.</span> <span style='background-color:rgba(135,206,250,0.002'>the</span> <span style='background-color:rgba(135,206,250,0.002'>special</span> <span style='background-color:rgba(135,206,250,0.003'>effects</span> <span style='background-color:rgba(135,206,250,0.001'>are</span> <span style='background-color:rgba(135,206,250,0.005'>awesome</span> <span style='background-color:rgba(135,206,250,0.001'>,</span> <span style='background-color:rgba(135,206,250,0.006'>but</span> <span style='background-color:rgba(135,206,250,0.004'>what</span> <span style='background-color:rgba(135,206,250,0.001'>is</span> <span style='background-color:rgba(135,206,250,0.004'>even</span> <span style='background-color:rgba(135,206,250,0.007'>better</span> <span style='background-color:rgba(135,206,250,0.003'>is</span> <span style='background-color:rgba(135,206,250,0.003'>the</span> <span style='background-color:rgba(135,206,250,0.009'>writing</span> <span style='background-color:rgba(135,206,250,0.002'>.</span> <span style='background-color:rgba(135,206,250,0.002'>you</span> <span style='background-color:rgba(135,206,250,0.001'>get</span> <span style='background-color:rgba(135,206,250,0.001'>a</span> <span style='background-color:rgba(135,206,250,0.001'>chance</span> <span style='background-color:rgba(135,206,250,0.000'>to</span> <span style='background-color:rgba(135,206,250,0.001'>learn</span> <span style='background-color:rgba(135,206,250,0.001'>more</span> <span style='background-color:rgba(135,206,250,0.000'>about</span> <span style='background-color:rgba(135,206,250,0.000'>our</span> <span style='background-color:rgba(135,206,250,0.000'>beloved</span> <span style='background-color:rgba(135,206,250,0.000'>doctor</span> <span style='background-color:rgba(135,206,250,0.001'>and</span> <span style='background-color:rgba(135,206,250,0.004'>maybe</span> <span style='background-color:rgba(135,206,250,0.001'>a</span> <span style='background-color:rgba(135,206,250,0.001'>bit</span> <span style='background-color:rgba(135,206,250,0.001'>of</span> <span style='background-color:rgba(135,206,250,0.001'>a</span> <span style='background-color:rgba(135,206,250,0.001'>reason</span> <span style='background-color:rgba(135,206,250,0.001'>why</span> <span style='background-color:rgba(135,206,250,0.000'>he</span> <span style='background-color:rgba(135,206,250,0.001'>loves</span> <span style='background-color:rgba(135,206,250,0.000'>the</span> <span style='background-color:rgba(135,206,250,0.001'>planet</span> <span style='background-color:rgba(135,206,250,0.001'>earth</span> <span style='background-color:rgba(135,206,250,0.000'>so</span> <span style='background-color:rgba(135,206,250,0.001'>much</span> <span style='background-color:rgba(135,206,250,0.008'>.</span> <span style='background-color:rgba(135,206,250,0.019'>without</span> <span style='background-color:rgba(135,206,250,0.004'>giving</span> <span style='background-color:rgba(135,206,250,0.000'>too</span> <span style='background-color:rgba(135,206,250,0.001'>much</span> <span style='background-color:rgba(135,206,250,0.001'>away</span> <span style='background-color:rgba(135,206,250,0.010'>,</span> <span style='background-color:rgba(135,206,250,0.017'>it</span> <span style='background-color:rgba(135,206,250,0.020'>is</span> <span style='background-color:rgba(135,206,250,0.017'>a</span> <span style='background-color:rgba(135,206,250,0.016'>very</span> <span style='background-color:rgba(135,206,250,0.041'>worth</span> <span style='background-color:rgba(135,206,250,0.029'>##while</span> <span style='background-color:rgba(135,206,250,0.025'>series</span> <span style='background-color:rgba(135,206,250,0.010'>to</span> <span style='background-color:rgba(135,206,250,0.015'>watch</span> <span style='background-color:rgba(135,206,250,0.019'>.</span> <span style='background-color:rgba(135,206,250,0.003'>christopher</span> <span style='background-color:rgba(135,206,250,0.001'>ec</span> <span style='background-color:rgba(135,206,250,0.001'>##cel</span> <span style='background-color:rgba(135,206,250,0.002'>##ston</span> <span style='background-color:rgba(135,206,250,0.006'>brought</span> <span style='background-color:rgba(135,206,250,0.003'>a</span> <span style='background-color:rgba(135,206,250,0.002'>side</span> <span style='background-color:rgba(135,206,250,0.001'>of</span> <span style='background-color:rgba(135,206,250,0.001'>the</span> <span style='background-color:rgba(135,206,250,0.004'>doctor</span> <span style='background-color:rgba(135,206,250,0.002'>that</span> <span style='background-color:rgba(135,206,250,0.002'>we</span> <span style='background-color:rgba(135,206,250,0.001'>never</span> <span style='background-color:rgba(135,206,250,0.001'>get</span> <span style='background-color:rgba(135,206,250,0.001'>to</span> <span style='background-color:rgba(135,206,250,0.001'>see</span> <span style='background-color:rgba(135,206,250,0.002'>,</span> <span style='background-color:rgba(135,206,250,0.005'>a</span> <span style='background-color:rgba(135,206,250,0.004'>bitter</span> <span style='background-color:rgba(135,206,250,0.004'>and</span> <span style='background-color:rgba(135,206,250,0.004'>angry</span> <span style='background-color:rgba(135,206,250,0.003'>one</span> <span style='background-color:rgba(135,206,250,0.013'>but</span> <span style='background-color:rgba(135,206,250,0.014'>yet</span> <span style='background-color:rgba(135,206,250,0.034'>lo</span> <span style='background-color:rgba(135,206,250,0.016'>##vable</span> <span style='background-color:rgba(135,206,250,0.003'>at</span> <span style='background-color:rgba(135,206,250,0.004'>the</span> <span style='background-color:rgba(135,206,250,0.013'>same</span> <span style='background-color:rgba(135,206,250,0.004'>time</span> <span style='background-color:rgba(135,206,250,0.013'>.</span> <span style='background-color:rgba(135,206,250,0.015'>a</span> <span style='background-color:rgba(135,206,250,0.031'>must</span> <span style='background-color:rgba(135,206,250,0.017'>see</span> <span style='background-color:rgba(135,206,250,0.012'>!</span> <span style='background-color:rgba(135,206,250,1.242'>[SEP]</span>

Explainer: attention_qk_softmax_trained


<span style='background-color:rgba(135,206,250,0.797'>[CLS]</span> <span style='background-color:rgba(135,206,250,0.002'>first</span> <span style='background-color:rgba(135,206,250,0.004'>off</span> <span style='background-color:rgba(135,206,250,0.002'>,</span> <span style='background-color:rgba(135,206,250,0.003'>i</span> <span style='background-color:rgba(135,206,250,0.006'>have</span> <span style='background-color:rgba(135,206,250,0.005'>been</span> <span style='background-color:rgba(135,206,250,0.004'>a</span> <span style='background-color:rgba(135,206,250,0.003'>fan</span> <span style='background-color:rgba(135,206,250,0.001'>of</span> <span style='background-color:rgba(135,206,250,0.001'>the</span> <span style='background-color:rgba(135,206,250,0.001'>show</span> <span style='background-color:rgba(135,206,250,0.006'>back</span> <span style='background-color:rgba(135,206,250,0.005'>when</span> <span style='background-color:rgba(135,206,250,0.003'>my</span> <span style='background-color:rgba(135,206,250,0.008'>pbs</span> <span style='background-color:rgba(135,206,250,0.004'>station</span> <span style='background-color:rgba(135,206,250,0.004'>started</span> <span style='background-color:rgba(135,206,250,0.002'>showing</span> <span style='background-color:rgba(135,206,250,0.002'>it</span> <span style='background-color:rgba(135,206,250,0.006'>back</span> <span style='background-color:rgba(135,206,250,0.005'>in</span> <span style='background-color:rgba(135,206,250,0.007'>1981</span> <span style='background-color:rgba(135,206,250,0.002'>.</span> <span style='background-color:rgba(135,206,250,0.002'>i</span> <span style='background-color:rgba(135,206,250,0.001'>learned</span> <span style='background-color:rgba(135,206,250,0.002'>many</span> <span style='background-color:rgba(135,206,250,0.001'>things</span> <span style='background-color:rgba(135,206,250,0.000'>about</span> <span style='background-color:rgba(135,206,250,0.001'>the</span> <span style='background-color:rgba(135,206,250,0.000'>show</span> <span style='background-color:rgba(135,206,250,0.000'>and</span> <span style='background-color:rgba(135,206,250,0.000'>the</span> <span style='background-color:rgba(135,206,250,0.000'>people</span> <span style='background-color:rgba(135,206,250,0.000'>who</span> <span style='background-color:rgba(135,206,250,0.001'>were</span> <span style='background-color:rgba(135,206,250,0.001'>in</span> <span style='background-color:rgba(135,206,250,0.002'>or</span> <span style='background-color:rgba(135,206,250,0.002'>contributed</span> <span style='background-color:rgba(135,206,250,0.002'>to</span> <span style='background-color:rgba(135,206,250,0.002'>the</span> <span style='background-color:rgba(135,206,250,0.003'>show</span> <span style='background-color:rgba(135,206,250,0.020'>.</span> <span style='background-color:rgba(135,206,250,0.116'>this</span> <span style='background-color:rgba(135,206,250,0.137'>latest</span> <span style='background-color:rgba(135,206,250,0.070'>installment</span> <span style='background-color:rgba(135,206,250,0.033'>of</span> <span style='background-color:rgba(135,206,250,0.073'>doctor</span> <span style='background-color:rgba(135,206,250,0.053'>who</span> <span style='background-color:rgba(135,206,250,0.149'>made</span> <span style='background-color:rgba(135,206,250,0.139'>a</span> <span style='background-color:rgba(135,206,250,0.302'>great</span> <span style='background-color:rgba(135,206,250,0.167'>impression</span> <span style='background-color:rgba(135,206,250,0.088'>on</span> <span style='background-color:rgba(135,206,250,0.062'>me</span> <span style='background-color:rgba(135,206,250,0.062'>.</span> <span style='background-color:rgba(135,206,250,0.026'>the</span> <span style='background-color:rgba(135,206,250,0.023'>original</span> <span style='background-color:rgba(135,206,250,0.015'>series</span> <span style='background-color:rgba(135,206,250,0.008'>,</span> <span style='background-color:rgba(135,206,250,0.007'>aka</span> <span style='background-color:rgba(135,206,250,0.007'>classic</span> <span style='background-color:rgba(135,206,250,0.013'>series</span> <span style='background-color:rgba(135,206,250,0.005'>,</span> <span style='background-color:rgba(135,206,250,0.018'>was</span> <span style='background-color:rgba(135,206,250,0.020'>made</span> <span style='background-color:rgba(135,206,250,0.007'>fun</span> <span style='background-color:rgba(135,206,250,0.012'>of</span> <span style='background-color:rgba(135,206,250,0.011'>by</span> <span style='background-color:rgba(135,206,250,0.013'>the</span> <span style='background-color:rgba(135,206,250,0.026'>bad</span> <span style='background-color:rgba(135,206,250,0.007'>special</span> <span style='background-color:rgba(135,206,250,0.008'>effects</span> <span style='background-color:rgba(135,206,250,0.011'>and</span> <span style='background-color:rgba(135,206,250,0.047'>/</span> <span style='background-color:rgba(135,206,250,0.018'>or</span> <span style='background-color:rgba(135,206,250,0.018'>wo</span> <span style='background-color:rgba(135,206,250,0.022'>##bbly</span> <span style='background-color:rgba(135,206,250,0.020'>sets</span> <span style='background-color:rgba(135,206,250,0.051'>.</span> <span style='background-color:rgba(135,206,250,0.412'>well</span> <span style='background-color:rgba(135,206,250,0.230'>,</span> <span style='background-color:rgba(135,206,250,0.411'>this</span> <span style='background-color:rgba(135,206,250,0.458'>is</span> <span style='background-color:rgba(135,206,250,1.250'>no</span> <span style='background-color:rgba(135,206,250,0.686'>more</span> <span style='background-color:rgba(135,206,250,0.785'>true</span> <span style='background-color:rgba(135,206,250,0.329'>.</span> <span style='background-color:rgba(135,206,250,0.091'>the</span> <span style='background-color:rgba(135,206,250,0.084'>special</span> <span style='background-color:rgba(135,206,250,0.124'>effects</span> <span style='background-color:rgba(135,206,250,0.077'>are</span> <span style='background-color:rgba(135,206,250,0.168'>awesome</span> <span style='background-color:rgba(135,206,250,0.034'>,</span> <span style='background-color:rgba(135,206,250,0.152'>but</span> <span style='background-color:rgba(135,206,250,0.071'>what</span> <span style='background-color:rgba(135,206,250,0.031'>is</span> <span style='background-color:rgba(135,206,250,0.068'>even</span> <span style='background-color:rgba(135,206,250,0.084'>better</span> <span style='background-color:rgba(135,206,250,0.036'>is</span> <span style='background-color:rgba(135,206,250,0.023'>the</span> <span style='background-color:rgba(135,206,250,0.026'>writing</span> <span style='background-color:rgba(135,206,250,0.011'>.</span> <span style='background-color:rgba(135,206,250,0.006'>you</span> <span style='background-color:rgba(135,206,250,0.004'>get</span> <span style='background-color:rgba(135,206,250,0.005'>a</span> <span style='background-color:rgba(135,206,250,0.004'>chance</span> <span style='background-color:rgba(135,206,250,0.002'>to</span> <span style='background-color:rgba(135,206,250,0.005'>learn</span> <span style='background-color:rgba(135,206,250,0.009'>more</span> <span style='background-color:rgba(135,206,250,0.004'>about</span> <span style='background-color:rgba(135,206,250,0.002'>our</span> <span style='background-color:rgba(135,206,250,0.003'>beloved</span> <span style='background-color:rgba(135,206,250,0.003'>doctor</span> <span style='background-color:rgba(135,206,250,0.007'>and</span> <span style='background-color:rgba(135,206,250,0.051'>maybe</span> <span style='background-color:rgba(135,206,250,0.027'>a</span> <span style='background-color:rgba(135,206,250,0.033'>bit</span> <span style='background-color:rgba(135,206,250,0.026'>of</span> <span style='background-color:rgba(135,206,250,0.029'>a</span> <span style='background-color:rgba(135,206,250,0.025'>reason</span> <span style='background-color:rgba(135,206,250,0.032'>why</span> <span style='background-color:rgba(135,206,250,0.017'>he</span> <span style='background-color:rgba(135,206,250,0.031'>loves</span> <span style='background-color:rgba(135,206,250,0.018'>the</span> <span style='background-color:rgba(135,206,250,0.015'>planet</span> <span style='background-color:rgba(135,206,250,0.017'>earth</span> <span style='background-color:rgba(135,206,250,0.015'>so</span> <span style='background-color:rgba(135,206,250,0.021'>much</span> <span style='background-color:rgba(135,206,250,0.025'>.</span> <span style='background-color:rgba(135,206,250,0.017'>without</span> <span style='background-color:rgba(135,206,250,0.005'>giving</span> <span style='background-color:rgba(135,206,250,0.002'>too</span> <span style='background-color:rgba(135,206,250,0.003'>much</span> <span style='background-color:rgba(135,206,250,0.003'>away</span> <span style='background-color:rgba(135,206,250,0.017'>,</span> <span style='background-color:rgba(135,206,250,0.019'>it</span> <span style='background-color:rgba(135,206,250,0.018'>is</span> <span style='background-color:rgba(135,206,250,0.014'>a</span> <span style='background-color:rgba(135,206,250,0.010'>very</span> <span style='background-color:rgba(135,206,250,0.025'>worth</span> <span style='background-color:rgba(135,206,250,0.016'>##while</span> <span style='background-color:rgba(135,206,250,0.013'>series</span> <span style='background-color:rgba(135,206,250,0.003'>to</span> <span style='background-color:rgba(135,206,250,0.005'>watch</span> <span style='background-color:rgba(135,206,250,0.012'>.</span> <span style='background-color:rgba(135,206,250,0.002'>christopher</span> <span style='background-color:rgba(135,206,250,0.001'>ec</span> <span style='background-color:rgba(135,206,250,0.001'>##cel</span> <span style='background-color:rgba(135,206,250,0.001'>##ston</span> <span style='background-color:rgba(135,206,250,0.009'>brought</span> <span style='background-color:rgba(135,206,250,0.006'>a</span> <span style='background-color:rgba(135,206,250,0.002'>side</span> <span style='background-color:rgba(135,206,250,0.001'>of</span> <span style='background-color:rgba(135,206,250,0.002'>the</span> <span style='background-color:rgba(135,206,250,0.003'>doctor</span> <span style='background-color:rgba(135,206,250,0.005'>that</span> <span style='background-color:rgba(135,206,250,0.004'>we</span> <span style='background-color:rgba(135,206,250,0.007'>never</span> <span style='background-color:rgba(135,206,250,0.009'>get</span> <span style='background-color:rgba(135,206,250,0.009'>to</span> <span style='background-color:rgba(135,206,250,0.007'>see</span> <span style='background-color:rgba(135,206,250,0.019'>,</span> <span style='background-color:rgba(135,206,250,0.070'>a</span> <span style='background-color:rgba(135,206,250,0.042'>bitter</span> <span style='background-color:rgba(135,206,250,0.070'>and</span> <span style='background-color:rgba(135,206,250,0.040'>angry</span> <span style='background-color:rgba(135,206,250,0.040'>one</span> <span style='background-color:rgba(135,206,250,0.075'>but</span> <span style='background-color:rgba(135,206,250,0.052'>yet</span> <span style='background-color:rgba(135,206,250,0.070'>lo</span> <span style='background-color:rgba(135,206,250,0.045'>##vable</span> <span style='background-color:rgba(135,206,250,0.017'>at</span> <span style='background-color:rgba(135,206,250,0.012'>the</span> <span style='background-color:rgba(135,206,250,0.024'>same</span> <span style='background-color:rgba(135,206,250,0.014'>time</span> <span style='background-color:rgba(135,206,250,0.027'>.</span> <span style='background-color:rgba(135,206,250,0.032'>a</span> <span style='background-color:rgba(135,206,250,0.073'>must</span> <span style='background-color:rgba(135,206,250,0.030'>see</span> <span style='background-color:rgba(135,206,250,0.028'>!</span> <span style='background-color:rgba(135,206,250,0.795'>[SEP]</span>

----------
